In [1]:
from spare.collection import  SparseCollectionCSR
import json
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("naver/splade-cocondenser-ensembledistil")
token_to_id = tokenizer.vocab


/home/tiagoalmeida/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
spare_col = SparseCollectionCSR.load_from_file("beir_datasets/msmarco/splade_msmarco_index_fp16")

matrix = spare_col.get_sparse_matrix().to("cuda")

/home/tiagoalmeida/safe_volume/sparse-retrieval/spare/backend_torch.py:102: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:54.)
  return torch.sparse_csr_tensor(crow_indices, col_indices, values, shape, dtype=self.types_converter[dtype])


In [3]:
questions = []
questions_ids = []
with open("beir_datasets/msmarco/splade_msmarco_questions_bow.jsonl") as f:
    for q in map(json.loads, f):
        indices, values = list(zip(*map(lambda x: (token_to_id[x[0]],x[1]),q["bow"].items())))
        indices = torch.tensor([indices], dtype=torch.int32)
        values = torch.tensor(values, dtype=torch.float16)       
        questions.append(torch.sparse_coo_tensor(indices, values.squeeze(0), (matrix.shape[-1],)).to_dense())
        questions_ids.append(q["docno"])

In [6]:
from tqdm import tqdm

for q in tqdm(questions):
    q = q.to("cuda")
    r = matrix @ q


KeyboardInterrupt: 

In [21]:
indices, values = list(zip(*map(lambda x: (token_to_id[x[0]],x[1]),questions[0].items())))

In [22]:
indices = torch.tensor([indices], dtype=torch.int32)
values = torch.tensor(values, dtype=torch.float16)

tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float16)

30522